## Import Libraries and Packages


In [2]:
pip install skillsnetwork

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [3]:
import skillsnetwork

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [6]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

<a id='item32'></a>


## Download Data


In [7]:
## get the data
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/concrete_data_week3.zip", overwrite=True)

  0%|          | 0/30036 [00:00<?, ?it/s]

Saved to '.'


<a id='item33'></a>


## Define Global Constants


In [8]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>


## Construct ImageDataGenerator Instances


In [9]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [10]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.


In [12]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 5001 images belonging to 2 classes.


<a id='item35'></a>


In [13]:
model = Sequential()

add the ResNet50 pre-trained model to out model. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.


In [14]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

94765736/94765736 [==============================] - 4s 0us/step


define output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.


In [15]:
model.add(Dense(num_classes, activation='softmax'))

access the model's layers using the *layers* attribute of our model object.


In [16]:
model.layers

our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.


access the ResNet50 layers by running the following:


In [17]:
model.layers[0].layers

to train only the dense output layer we run the following


In [18]:
model.layers[0].trainable = False

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


compile the model using the **adam** optimizer.


In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

define steps per epoch as follows:


In [21]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [22]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-22-172e67583a70>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
101/101 [==============================] - 2863s 28s/step - loss: 0.0517 - accuracy: 0.9800 - val_loss: 0.0125 - val_accuracy: 0.9958
Epoch 2/2
101/101 [==============================] - 2817s 28s/step - loss: 0.0067 - accuracy: 0.9989 - val_loss: 0.0087 - val_accuracy: 0.9970


In [23]:
model.save('classifier_resnet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
model.save('my_model.keras')